In [113]:
import pandas as pd
import re
import ast
from openpyxl import load_workbook
import urllib.parse as ul

In [135]:
liste_article_octobre = pd.read_excel(r'C:\Users\Hanaa\Desktop\PETITFUTE\data\exports\listes_article_octobre.xlsx')
stay22 = pd.read_csv(r'C:\Users\Hanaa\Desktop\PETITFUTE\data\exports\Bookings from 2025-10-01 to 2025-10-29.csv',sep=',', index_col=False)
gyg = pd.read_excel(r'C:\Users\Hanaa\Desktop\PETITFUTE\data\exports\get-your-guide-gxcSWuH9.xlsx')
booking = pd.read_csv(r'C:\Users\Hanaa\Desktop\PETITFUTE\data\exports\bookingcom-Fp693Iv8.csv')


c:\Users\Hanaa\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [136]:
liste_article_octobre['article_id'] = liste_article_octobre['URL'].str.extract(r'/(a\d+)-')

# MATCHING DIRECT

## STAY22

Ne garder que les liens petitfute

In [137]:
stay22 = stay22[stay22['Campaign IDs'].fillna('').str.startswith("petitfute, a")]

# Extraction de l'id article 

stay22['article_id'] = stay22['Campaign IDs'].str.extract(r'(a\d+)')

In [138]:
# Agrégation des commissions par article_id
commission_sum = stay22.groupby('article_id', as_index=False)['Commission'].sum()
commission_sum = commission_sum.rename(columns={'Commission': 'STAY22_COMMISSION'})


# Left join sur article_id
merged = liste_article_octobre.merge(
    commission_sum,
    on='article_id',
    how='left'
)

In [139]:
merged = merged[['Ville','Mois','URL','article_id','STAY22_COMMISSION']]

## OPTI DIGITAL

In [ ]:
# ===  Charger le fichier Excel avec les formules  ===
wb = load_workbook(
    r"C:\Users\Hanaa\Desktop\PETITFUTE\data\exports\Opti Digital Ad Manager Hub v5.0 - Petit Futé_Audience Analytics_Tableau.xlsx",
    data_only=False
)
ws = wb.active

# ===  Extraire les URL ===
urls = []
for row in ws.iter_rows(min_col=3, max_col=3):  # colonne C
    for cell in row:
        if isinstance(cell.value, str) and cell.value.startswith('=HYPERLINK'):
            match = re.search(r'"(https?://[^"]+)"', cell.value)
            url = match.group(1) if match else None
            if isinstance(url, str) and "google.com/url" in url:
                m = re.search(r"q=(https[^&]+)", url)
                if m:
                    url = ul.unquote(m.group(1))
            urls.append(url)
        else:
            urls.append(cell.value)

# --- Enlever la première ligne (header Excel) ---
urls = urls[1:]

# ===  Charger avec pandas ===
opti = pd.read_excel(
    r"C:\Users\Hanaa\Desktop\PETITFUTE\data\exports\Opti Digital Ad Manager Hub v5.0 - Petit Futé_Audience Analytics_Tableau.xlsx"
)

# ===  Remplacer la colonne URL par les liens propres ===
opti["URL"] = urls

# ===  Sélectionner les colonnes d’intérêt ===
opti = opti[['Breakdown', 'URL', 'Net Revenue', 'Page Views', 'Pages RPM', 'Net eCPM', 'Prebid eCPM']]
opti['article_id'] = opti['URL'].str.extract(r'/(a\d+)-')

In [141]:
opti = opti[opti['article_id'].notna()]

# Agrégation des commissions par article_id
commission_sum_opti = opti.groupby('article_id', as_index=False)['Net Revenue'].sum()
commission_sum_opti = commission_sum_opti.rename(columns={'Net Revenue': 'OPTI_COMMISSION'})


# Left join sur article_id
merged = merged.merge(
    commission_sum_opti,
    on='article_id',
    how='left'
)

In [ ]:
merged

np.float64(13144.24)

# MATCHING INDIRECT

Création de la liste d'url de la base (ici octobre)

In [147]:
base_URL = liste_article_octobre[['URL']]

Code permettant pour chaque URL de la base de récupérer tous les liens booking et gyg présents
- **Entrée** : base_URL 
    - contient tous les url de la base du mois
- **Sortie** : df_final 
    - Contient pour chaque url de la base du mois  :  
        - une liste des url booking 
        - une liste des url gyg

In [ ]:
#!python scraping_petitfute.py

Import du fichier généré

In [90]:
df_final = pd.read_excel(r"C:\Users\Hanaa\Desktop\PETITFUTE\data\outputs\liens_partenaire_octobre.xlsx")

# Forcer en listes (et non str)
df_final["getyourguide_links"] = df_final["getyourguide_links"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

## GET YOUR GUIDE

In [91]:
def extract_gyg_ids_from_list(links):
    """Extrait tous les identifiants GetYourGuide (t + 5 ou 6 chiffres) d'une liste d'URLs."""
    if not isinstance(links, list):
        return []
    ids = []
    for url in links:
        # Cherche un motif 't' suivi de 5 ou 6 chiffres
        match = re.search(r"t\d{5,6}", url)
        if match:
            ids.append(match.group(0))
    return ids

In [92]:
# ✅ Appliquer la fonction à ta colonne existante
df_final["id_getyourguide_link"] = df_final["getyourguide_links"].apply(extract_gyg_ids_from_list)

In [93]:
df_final 

,URL,getyourguide_links,booking_links,id_getyourguide_link
0,https://www.petitfute.com/v62250-kew/actualite...,[https://www.getyourguide.fr/londres-l57/londr...,['https://www.booking.com/hotel/gb/ibis-london...,"[t41419, t129039, t31392, t310718, t34660, t38..."
1,https://www.petitfute.com/v48299-bangli/actual...,[https://www.getyourguide.fr/seminyak-l95096/k...,['https://www.booking.com/hotel/id/sanak-retre...,"[t791280, t162114, t393678, t187641, t55668, t..."
2,https://www.petitfute.com/v43490-caldera-de-ta...,[https://www.getyourguide.fr/tenerife-l350/vis...,[],"[t51524, t417569, t206580, t58006, t534639, t1..."
3,https://www.petitfute.com/v49846-genes/actuali...,[https://www.getyourguide.fr/genes-l1009/visit...,['https://www.booking.com/hotel/it/ostellin-ge...,"[t622176, t13191, t50734, t29705, t399086, t81..."
4,https://www.petitfute.com/v48986-copenhague/ac...,[https://www.getyourguide.fr/faelledparken-l12...,['https://www.booking.com/hotel/dk/steel-house...,"[t72048, t66660, t504350, t168407, t37848, t22..."
...,...,...,...,...
94,https://www.petitfute.com/v42470-bodrum/actual...,[https://www.getyourguide.fr/bodrum-l846/tour-...,['https://www.booking.com/hotel/tr/seckin-kona...,"[t653504, t103136, t476435, t467340]"
95,https://www.petitfute.com/v42420-antalya/actua...,[https://www.getyourguide.fr/antalya-l172/anta...,['https://www.booking.com/hotel/tr/argos.fr.ht...,"[t950470, t469125, t468561, t975032, t949847, ..."
96,https://www.petitfute.com/v41475-chiang-rai/ac...,[https://www.getyourguide.fr/bangkok-l169/bang...,['https://www.booking.com/hotel/th/baan-noppaw...,"[t417016, t425854, t128406, t417017, t52629, t..."
97,https://www.petitfute.com/v48393-seoul/actuali...,[https://www.getyourguide.fr/seoul-l197/palais...,['https://www.booking.com/hotel/kr/grand-hyatt...,"[t467593, t63107, t113551, t492074, t440500, t..."


In [94]:
gyg_activity = gyg[['View activity', 'Potential income']]

In [95]:
def extract_gyg_id(url):
    """Retourne l’identifiant GetYourGuide (t + 5 ou 6 chiffres) trouvé dans l’URL."""
    if not isinstance(url, str):
        return None
    match = re.search(r"t\d{5,6}", url)
    return match.group(0) if match else None

# Appliquer la fonction
gyg_activity["id_gyg"] = gyg_activity["View activity"].apply(extract_gyg_id)

C:\Users\Hanaa\AppData\Local\Temp\ipykernel_17716\426062652.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gyg_activity["id_gyg"] = gyg_activity["View activity"].apply(extract_gyg_id)


In [97]:
gyg_activity["Potential income"] = pd.to_numeric(
    gyg_activity["Potential income"], errors="coerce"
).fillna(0)


C:\Users\Hanaa\AppData\Local\Temp\ipykernel_17716\2979681287.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gyg_activity["Potential income"] = pd.to_numeric(


In [98]:
# --- Fonction pour calculer la somme du revenu GYG par ligne ---
def calc_revenu_gyg(ids, df_ref):
    """Somme des potential_income pour une liste d'IDs à partir du df_ref."""
    if not isinstance(ids, list) or len(ids) == 0:
        return 0.0
    subset = df_ref[df_ref["id_gyg"].isin(ids)]
    return subset["Potential income"].sum()

# --- Application à la table Petit Futé ---
df_final["revenu_gyg"] = df_final["id_getyourguide_link"].apply(lambda ids: calc_revenu_gyg(ids, gyg_activity))



In [100]:
print(df_final['revenu_gyg'].sum())
df_final['revenu_gyg'].value_counts()


172.92000000000002


revenu_gyg
0.00     86
6.72      2
6.45      1
7.51      1
4.06      1
71.68     1
2.80      1
4.48      1
10.50     1
8.40      1
31.36     1
11.40     1
0.84      1
Name: count, dtype: int64

In [101]:
df_final

,URL,getyourguide_links,booking_links,id_getyourguide_link,revenu_gyg
0,https://www.petitfute.com/v62250-kew/actualite...,[https://www.getyourguide.fr/londres-l57/londr...,['https://www.booking.com/hotel/gb/ibis-london...,"[t41419, t129039, t31392, t310718, t34660, t38...",6.45
1,https://www.petitfute.com/v48299-bangli/actual...,[https://www.getyourguide.fr/seminyak-l95096/k...,['https://www.booking.com/hotel/id/sanak-retre...,"[t791280, t162114, t393678, t187641, t55668, t...",0.00
2,https://www.petitfute.com/v43490-caldera-de-ta...,[https://www.getyourguide.fr/tenerife-l350/vis...,[],"[t51524, t417569, t206580, t58006, t534639, t1...",0.00
3,https://www.petitfute.com/v49846-genes/actuali...,[https://www.getyourguide.fr/genes-l1009/visit...,['https://www.booking.com/hotel/it/ostellin-ge...,"[t622176, t13191, t50734, t29705, t399086, t81...",0.00
4,https://www.petitfute.com/v48986-copenhague/ac...,[https://www.getyourguide.fr/faelledparken-l12...,['https://www.booking.com/hotel/dk/steel-house...,"[t72048, t66660, t504350, t168407, t37848, t22...",0.00
...,...,...,...,...,...
94,https://www.petitfute.com/v42470-bodrum/actual...,[https://www.getyourguide.fr/bodrum-l846/tour-...,['https://www.booking.com/hotel/tr/seckin-kona...,"[t653504, t103136, t476435, t467340]",0.00
95,https://www.petitfute.com/v42420-antalya/actua...,[https://www.getyourguide.fr/antalya-l172/anta...,['https://www.booking.com/hotel/tr/argos.fr.ht...,"[t950470, t469125, t468561, t975032, t949847, ...",0.00
96,https://www.petitfute.com/v41475-chiang-rai/ac...,[https://www.getyourguide.fr/bangkok-l169/bang...,['https://www.booking.com/hotel/th/baan-noppaw...,"[t417016, t425854, t128406, t417017, t52629, t...",0.00
97,https://www.petitfute.com/v48393-seoul/actuali...,[https://www.getyourguide.fr/seoul-l197/palais...,['https://www.booking.com/hotel/kr/grand-hyatt...,"[t467593, t63107, t113551, t492074, t440500, t...",0.00


# BOOKING

In [16]:
base_URL

,URL
0,https://www.petitfute.com/v62250-kew/actualite...
1,https://www.petitfute.com/v48299-bangli/actual...
2,https://www.petitfute.com/v43490-caldera-de-ta...
3,https://www.petitfute.com/v49846-genes/actuali...
4,https://www.petitfute.com/v48986-copenhague/ac...
...,...
94,https://www.petitfute.com/v42470-bodrum/actual...
95,https://www.petitfute.com/v42420-antalya/actua...
96,https://www.petitfute.com/v41475-chiang-rai/ac...
97,https://www.petitfute.com/v48393-seoul/actuali...


In [17]:
df_liens = pd.read_excel(r"C:\Users\Hanaa\Desktop\PETITFUTE\data\outputs\liens_partenaire_octobre.xlsx")
df_liens.head()

,URL,getyourguide_links,booking_links
0,https://www.petitfute.com/v62250-kew/actualite...,['https://www.getyourguide.fr/londres-l57/lond...,['https://www.booking.com/hotel/gb/ibis-london...
1,https://www.petitfute.com/v48299-bangli/actual...,['https://www.getyourguide.fr/seminyak-l95096/...,['https://www.booking.com/hotel/id/sanak-retre...
2,https://www.petitfute.com/v43490-caldera-de-ta...,['https://www.getyourguide.fr/tenerife-l350/vi...,[]
3,https://www.petitfute.com/v49846-genes/actuali...,['https://www.getyourguide.fr/genes-l1009/visi...,['https://www.booking.com/hotel/it/ostellin-ge...
4,https://www.petitfute.com/v48986-copenhague/ac...,['https://www.getyourguide.fr/faelledparken-l1...,['https://www.booking.com/hotel/dk/steel-house...


In [34]:
df_liens

,URL,getyourguide_links,booking_links,type
0,https://www.petitfute.com/v62250-kew/actualite...,['https://www.getyourguide.fr/londres-l57/lond...,['https://www.booking.com/hotel/gb/ibis-london...,None
1,https://www.petitfute.com/v48299-bangli/actual...,['https://www.getyourguide.fr/seminyak-l95096/...,['https://www.booking.com/hotel/id/sanak-retre...,None
2,https://www.petitfute.com/v43490-caldera-de-ta...,['https://www.getyourguide.fr/tenerife-l350/vi...,[],None
3,https://www.petitfute.com/v49846-genes/actuali...,['https://www.getyourguide.fr/genes-l1009/visi...,['https://www.booking.com/hotel/it/ostellin-ge...,None
4,https://www.petitfute.com/v48986-copenhague/ac...,['https://www.getyourguide.fr/faelledparken-l1...,['https://www.booking.com/hotel/dk/steel-house...,None
...,...,...,...,...
94,https://www.petitfute.com/v42470-bodrum/actual...,['https://www.getyourguide.fr/bodrum-l846/tour...,['https://www.booking.com/hotel/tr/seckin-kona...,None
95,https://www.petitfute.com/v42420-antalya/actua...,['https://www.getyourguide.fr/antalya-l172/ant...,['https://www.booking.com/hotel/tr/argos.fr.ht...,None
96,https://www.petitfute.com/v41475-chiang-rai/ac...,['https://www.getyourguide.fr/bangkok-l169/ban...,['https://www.booking.com/hotel/th/baan-noppaw...,None
97,https://www.petitfute.com/v48393-seoul/actuali...,['https://www.getyourguide.fr/seoul-l197/palai...,['https://www.booking.com/hotel/kr/grand-hyatt...,None


In [35]:
type(df_liens['booking_links'].iloc[0])


str

In [38]:
import pandas as pd
import re
import ast

# --- 1️⃣ Charger uniquement les colonnes utiles ---
df_liens_tmp = df_liens[['URL', 'booking_links']].copy()

# --- 2️⃣ Convertir les chaînes en vraies listes (si besoin) ---
df_liens_tmp['booking_links'] = df_liens_tmp['booking_links'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else x
)

# --- 3️⃣ Exploser les listes pour avoir une ligne par lien ---
df_tmp = df_liens_tmp.explode('booking_links').reset_index(drop=True)

# --- 4️⃣ Supprimer les lignes sans lien ---
df_tmp = df_tmp[df_tmp['booking_links'].notna()].copy()

# --- 5️⃣ Extraire le type (ex : hotel, apartment, etc.) ---
def extraire_type_booking(url):
    if isinstance(url, str):
        match = re.search(r'booking\.com/([^/]+)/', url)
        if match:
            return match.group(1)
    return None

df_tmp['type'] = df_tmp['booking_links'].apply(extraire_type_booking)

# --- 6️⃣ Extraire le nom d’établissement (ex : park-hyatt-zurich) ---
def extraire_nom_hotel(url):
    if isinstance(url, str):
        match = re.search(r'booking\.com/hotel/[a-z]{2}/([^/.]+)', url)
        if match:
            return match.group(1)
    return None

df_tmp['nom_hotel'] = df_tmp['booking_links'].apply(extraire_nom_hotel)

# --- 7️⃣ Vérification du résultat ---
df_tmp[['URL', 'booking_links', 'type', 'nom_hotel']].head()


,URL,booking_links,type,nom_hotel
0,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/ibis-london-b...,hotel,ibis-london-blackfriars
1,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/ibis-london-b...,hotel,ibis-london-blackfriars
2,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/therookery.fr...,hotel,therookery
3,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/sanderson.fr....,hotel,sanderson
4,https://www.petitfute.com/v48299-bangli/actual...,https://www.booking.com/hotel/id/sanak-retreat...,hotel,sanak-retreat-bali


In [19]:
df_tmp = pd.read_excel(r'C:\Users\Hanaa\Desktop\PETITFUTE\data\outputs\booking_h2_exploded.xlsx')

In [20]:
df_tmp

,URL,booking_links,balise_h2
0,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/ibis-london-b...,ibis London Blackfriars
1,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/ibis-london-b...,ibis London Blackfriars
2,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/therookery.fr...,The Rookery
3,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/sanderson.fr....,Sanderson London
4,https://www.petitfute.com/v48299-bangli/actual...,https://www.booking.com/hotel/id/sanak-retreat...,Sanak Retreat Bali
...,...,...,...
274,https://www.petitfute.com/v52913-varsovie-wars...,https://www.booking.com/hotel/pl/novotelwarsza...,Novotel Warszawa Centrum
275,https://www.petitfute.com/v52913-varsovie-wars...,https://www.booking.com/hotel/pl/sofitelvictor...,Sofitel Warsaw Victoria
276,https://www.petitfute.com/v52913-varsovie-wars...,https://www.booking.com/hotel/pl/sofitelvictor...,Sofitel Warsaw Victoria
277,https://www.petitfute.com/v52913-varsovie-wars...,https://www.booking.com/hotel/pl/holiday-inn-w...,Mercure Warszawa Centrum


In [ ]:
https://www.booking.com/hotel/pl/novotelwarszawacentrumwarszawa.fr.html?aid=2324560&no_rooms=1&group_adults=2&label=actu-quefaire-varsovie

In [45]:
df_tmp[df_tmp['nom_hotel'].str.lower().str.contains('conquistador')]

,URL,booking_links,type,nom_hotel


In [30]:
booking

,Booking date,Booking number,Check-in date,Check-out date,Length of stay,Booking window,Status,Your commission,Commission %,Credit slip,...,Property type,Country,City,UFI,Affiliate ID,Label,Booker country,Booker language,User device,Travel purpose
0,10/8/2025,5362867269,10/27/2025,10/28/2025,1,19,Booked,3.56,40.0,NaN,...,Holiday home,FR,Franquevielle,-1428038,1180643,petitfute-BoxOSV-POI,France,FR,Mobile Browser,LEISURE
1,10/8/2025,5223394350,11/8/2025,11/11/2025,3,31,Booked,20.13,40.0,NaN,...,Hotel,CZ,Prague,-553173,2324560,actu-3jours-prague,Belgium,FR,Desktop,LEISURE
2,10/8/2025,5784114772,12/17/2025,12/19/2025,2,70,Booked,9.81,40.0,NaN,...,Lodge,CR,San Gerardo de Dota,900052668,1180643,conf_email_print,France,FR,Desktop,UNKNOWN
3,10/8/2025,5804108674,12/12/2025,12/14/2025,2,65,Booked,10.74,40.0,NaN,...,Hotel,CR,Dominical,-1107038,1180643,conf_email_print,France,FR,Desktop,UNKNOWN
4,10/8/2025,6162599111,12/10/2025,12/12/2025,2,63,Booked,7.07,40.0,NaN,...,Bed and Breakfast,CR,Quepos,-1108763,1180643,conf_email_print,France,FR,Desktop,UNKNOWN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,10/1/2025,5299601560,10/1/2025,10/2/2025,1,0,Booked,10.54,40.0,NaN,...,Hotel,ES,Córdoba,-378765,2324560,actu_quefaire_cordoue,France,FR,App,UNKNOWN
85,10/1/2025,5128273214,10/5/2025,10/6/2025,1,4,Booked,7.75,40.0,NaN,...,Hotel,FR,Saumur,-1469710,1180643,petitfute-bouton_reserver-Autres,Canada,FR,Tablet,LEISURE
86,10/1/2025,5905593911,10/2/2025,10/5/2025,3,1,Booked,10.50,40.0,NaN,...,Hostel,NL,Amsterdam,-2140479,2324560,amsterdam_3_jours,France,FR,App,LEISURE
87,10/1/2025,6755683486,10/1/2025,10/2/2025,1,0,Booked,8.16,40.0,NaN,...,Hotel,FR,Vannes,-1474712,1180643,vizity,France,FR,Desktop,LEISURE


In [157]:
df_final[df_final['URL']=='https://www.petitfute.com/v62250-kew/actualite/m17-top-10-insolites-voyage/a11635-que-faire-a-londres-a-noel-les-14-activites-incontournables.html']

,URL,getyourguide_links,booking_links,id_getyourguide_link,revenu_gyg
0,https://www.petitfute.com/v62250-kew/actualite...,[https://www.getyourguide.fr/londres-l57/londr...,['https://www.booking.com/hotel/gb/ibis-london...,"[t41419, t129039, t31392, t310718, t34660, t38...",6.45


In [23]:
import unicodedata

def normaliser_nom(nom):
    if not isinstance(nom, str):
        return ""
    # mettre en minuscules
    nom = nom.lower()
    # remplacer les tirets par des espaces
    nom = nom.replace("-", " ")
    # retirer accents
    nom = ''.join(
        c for c in unicodedata.normalize('NFD', nom)
        if unicodedata.category(c) != 'Mn'
    )
    # retirer espaces multiples
    nom = re.sub(r'\s+', ' ', nom).strip()
    return nom

df_tmp['nom_hotel_norm'] = df_tmp['balise_h2'].apply(normaliser_nom)
booking['property_name_norm'] = booking['Property name'].apply(normaliser_nom)


In [ ]:
df_jointure = df_tmp.merge(
    booking,
    left_on='nom_hotel_norm',
    right_on='property_name_norm',
    how='left',
    indicator=True
)
df_jointure

In [47]:
from rapidfuzz import process, fuzz

# --- 1️⃣ Liste des noms Booking normalisés ---
booking_names = booking['property_name_norm'].dropna().unique().tolist()

# --- 2️⃣ Fonction de recherche du meilleur match ---
def trouver_meilleur_match(nom):
    if not isinstance(nom, str) or not nom:
        return None, 0
    match, score, _ = process.extractOne(
        nom,
        booking_names,
        scorer=fuzz.token_sort_ratio  # bon pour les noms avec mots inversés
    )
    return match, score

# --- 3️⃣ Application sur df_tmp ---
df_tmp[['match_booking', 'score_match']] = df_tmp['nom_hotel_norm'].apply(
    lambda x: pd.Series(trouver_meilleur_match(x))
)

# --- 4️⃣ Vérification ---
df_tmp[['nom_hotel_norm', 'match_booking', 'score_match']].sort_values('score_match', ascending=False).head(20)
df_tmp

,URL,booking_links,balise_h2,nom_hotel_norm,match_booking,score_match
0,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/ibis-london-b...,ibis London Blackfriars,ibis london blackfriars,villas paradise arenal volcano lake view,44.444444
1,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/ibis-london-b...,ibis London Blackfriars,ibis london blackfriars,villas paradise arenal volcano lake view,44.444444
2,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/therookery.fr...,The Rookery,the rookery,the people lyon,53.846154
3,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/sanderson.fr....,Sanderson London,sanderson london,eurostars conquistador,47.368421
4,https://www.petitfute.com/v48299-bangli/actual...,https://www.booking.com/hotel/id/sanak-retreat...,Sanak Retreat Bali,sanak retreat bali,hotel blue sea,50.000000
...,...,...,...,...,...,...
274,https://www.petitfute.com/v52913-varsovie-wars...,https://www.booking.com/hotel/pl/novotelwarsza...,Novotel Warszawa Centrum,novotel warszawa centrum,hotel ibis cognac centre,54.166667
275,https://www.petitfute.com/v52913-varsovie-wars...,https://www.booking.com/hotel/pl/sofitelvictor...,Sofitel Warsaw Victoria,sofitel warsaw victoria,hotel villas rio mar,46.511628
276,https://www.petitfute.com/v52913-varsovie-wars...,https://www.booking.com/hotel/pl/sofitelvictor...,Sofitel Warsaw Victoria,sofitel warsaw victoria,hotel villas rio mar,46.511628
277,https://www.petitfute.com/v52913-varsovie-wars...,https://www.booking.com/hotel/pl/holiday-inn-w...,Mercure Warszawa Centrum,mercure warszawa centrum,eurostars conquistador,47.826087


In [46]:
from rapidfuzz import process, fuzz

booking_names = booking['property_name_norm'].dropna().unique().tolist()

def trouver_meilleur_match(nom):
    if not isinstance(nom, str) or not nom:
        return None, 0
    match, score, _ = process.extractOne(
        nom,
        booking_names,
        scorer=fuzz.partial_ratio  # bon pour les sous-chaînes
    )
    return match, score

df_tmp[['match_booking', 'score_match']] = df_tmp['nom_hotel_norm'].apply(
    lambda x: pd.Series(trouver_meilleur_match(x))
)

df_tmp[['nom_hotel_norm', 'match_booking', 'score_match']].sort_values('score_match', ascending=False).head(20)
df_tmp


,URL,booking_links,balise_h2,nom_hotel_norm,match_booking,score_match
0,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/ibis-london-b...,ibis London Blackfriars,ibis london blackfriars,hotel ibis cognac centre,48.780488
1,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/ibis-london-b...,ibis London Blackfriars,ibis london blackfriars,hotel ibis cognac centre,48.780488
2,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/therookery.fr...,The Rookery,the rookery,hotel royal,60.000000
3,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/sanderson.fr....,Sanderson London,sanderson london,tucantico lodge monteverde,56.250000
4,https://www.petitfute.com/v48299-bangli/actual...,https://www.booking.com/hotel/id/sanak-retreat...,Sanak Retreat Bali,sanak retreat bali,ty avel,61.538462
...,...,...,...,...,...,...
274,https://www.petitfute.com/v52913-varsovie-wars...,https://www.booking.com/hotel/pl/novotelwarsza...,Novotel Warszawa Centrum,novotel warszawa centrum,hostel,66.666667
275,https://www.petitfute.com/v52913-varsovie-wars...,https://www.booking.com/hotel/pl/sofitelvictor...,Sofitel Warsaw Victoria,sofitel warsaw victoria,hostel,66.666667
276,https://www.petitfute.com/v52913-varsovie-wars...,https://www.booking.com/hotel/pl/sofitelvictor...,Sofitel Warsaw Victoria,sofitel warsaw victoria,hostel,66.666667
277,https://www.petitfute.com/v52913-varsovie-wars...,https://www.booking.com/hotel/pl/holiday-inn-w...,Mercure Warszawa Centrum,mercure warszawa centrum,nh genova centro,56.000000


In [48]:
from rapidfuzz import process, fuzz

booking_names = booking['property_name_norm'].dropna().unique().tolist()

def trouver_meilleur_match(nom):
    if not isinstance(nom, str) or not nom:
        return None, 0
    match, score, _ = process.extractOne(
        nom,
        booking_names,
        scorer=fuzz.WRatio  # mélange plusieurs logiques
    )
    return match, score

df_tmp[['match_booking', 'score_match']] = df_tmp['nom_hotel_norm'].apply(
    lambda x: pd.Series(trouver_meilleur_match(x))
)

df_tmp[['nom_hotel_norm', 'match_booking', 'score_match']].sort_values('score_match', ascending=False).head(20)
df_tmp


,URL,booking_links,balise_h2,nom_hotel_norm,match_booking,score_match
0,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/ibis-london-b...,ibis London Blackfriars,ibis london blackfriars,los lagos lodge,44.608696
1,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/ibis-london-b...,ibis London Blackfriars,ibis london blackfriars,los lagos lodge,44.608696
2,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/therookery.fr...,The Rookery,the rookery,hotel royal,54.545455
3,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/sanderson.fr....,Sanderson London,sanderson london,tucantico lodge monteverde,50.625000
4,https://www.petitfute.com/v48299-bangli/actual...,https://www.booking.com/hotel/id/sanak-retreat...,Sanak Retreat Bali,sanak retreat bali,ty avel,55.384615
...,...,...,...,...,...,...
274,https://www.petitfute.com/v52913-varsovie-wars...,https://www.booking.com/hotel/pl/novotelwarsza...,Novotel Warszawa Centrum,novotel warszawa centrum,hotel novotel brussels off grand place,85.500000
275,https://www.petitfute.com/v52913-varsovie-wars...,https://www.booking.com/hotel/pl/sofitelvictor...,Sofitel Warsaw Victoria,sofitel warsaw victoria,hostel,60.000000
276,https://www.petitfute.com/v52913-varsovie-wars...,https://www.booking.com/hotel/pl/sofitelvictor...,Sofitel Warsaw Victoria,sofitel warsaw victoria,hostel,60.000000
277,https://www.petitfute.com/v52913-varsovie-wars...,https://www.booking.com/hotel/pl/holiday-inn-w...,Mercure Warszawa Centrum,mercure warszawa centrum,nh genova centro,50.400000


In [ ]:
df_tmp[df_tmp['nom_hotel_norm'].str.contains('sanderson')]

,URL,booking_links,balise_h2,nom_hotel_norm,match_booking,score_match
3,https://www.petitfute.com/v62250-kew/actualite...,https://www.booking.com/hotel/gb/sanderson.fr....,Sanderson London,sanderson london,eurostars conquistador,47.368421


In [32]:
df_tmp[df_tmp['booking_links']=='https://www.booking.com/hotel/gb/sanderson.fr.html?aid=2324560&no_rooms=1&group_adults=2&label=actu-londres-noel']
df_tmp[df_tmp['match_booking'].str.contains('sanderson')]
booking[booking['Property name'].str.lower().str.contains('sanderson')]


,Booking date,Booking number,Check-in date,Check-out date,Length of stay,Booking window,Status,Your commission,Commission %,Credit slip,...,Country,City,UFI,Affiliate ID,Label,Booker country,Booker language,User device,Travel purpose,property_name_norm


In [33]:
print(df_jointure['_merge'].value_counts()
)

_merge
left_only     277
both            2
right_only      0
Name: count, dtype: int64
